## Workflow

(for v0.3.0)
https://github.com/ASFHyP3/hyp3-isce2/releases/tag/v0.3.0

https://github.com/ASFHyP3/hyp3-isce2/tree/v0.3.0


1. Find a burst to process
2. Search ASF for scenes
3. Select dates to process
4. Create processing subfolders to execute burst processing in

In [1]:
import geopandas as gpd
import asf_search as asf
import requests
import os
# not sure why DEBUG statements is happening in other libraries if imported after hyp3_isce2...
import logging
import hyp3_isce2.burst as hb # https://github.com/ASFHyP3/hyp3-isce2/issues/53 ?
rootlogs = logging.getLogger()
rootlogs.setLevel('WARNING')
import numpy as np
import xmlschema
import lxml
import re
import shapely

In [2]:
import hyp3_isce2
hyp3_isce2.__version__ # syntax changing rapidly, so stick with 0.3.0

'0.7.1'

In [3]:
# https://geojson.io/#new&map=15.23/47.654452/-122.303174
gfa = gpd.GeoDataFrame.from_features( {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          108.172,
          22.60
        ],
        "type": "Point"
      }
    }
  ]
},
 crs=4326   
)
gfa.explore()

## Find a single burst

1. You need to download the burst database

https://sar-mpc.eu/test-data-sets/

Sentinel-1 Burst ID Map, version 20220530, generated by the SAR-MPC service, available on Sentinel-1 MPC Test data sets

In [4]:
# For now consider selecting a burst covering a point
gf = gpd.read_file('burst_map_IW_000001_375887.sqlite3',
                   mask=gfa)

In [5]:
gf.head()

,burst_id,subswath_name,relative_orbit_number,time_from_anx_sec,orbit_pass,geometry
0,37457,IW2,18,2599.290518,DESCENDING,"MULTIPOLYGON Z (((108.62764 22.56485 0.00000, ..."
1,116116,IW3,55,354.217610,ASCENDING,"MULTIPOLYGON Z (((107.56952 22.32915 0.00000, ..."
2,116117,IW3,55,356.975883,ASCENDING,"MULTIPOLYGON Z (((107.53634 22.49554 0.00000, ..."
3,335208,IW1,157,361.558646,ASCENDING,"MULTIPOLYGON Z (((107.89361 22.49101 0.00000, ..."


In [6]:
gf.explore()

In [7]:
# Ascending test
burstId = '157_IW1_335208'

relorb = int(burstId.split('_')[0])
subswath = burstId.split('_')[1]
idnum = int(burstId.split('_')[2])

ind = (gf.relative_orbit_number == relorb) & (gf.subswath_name == subswath) & (gf.burst_id == idnum)

myburst = gf[ind]
#myburst
print(myburst.geometry.values[0].centroid)
print(myburst.geometry.values[0])


POINT (108.28868724595392 22.663049108771247)
MULTIPOLYGON Z (((107.893614 22.491012 0, 108.321901 22.565881 0, 108.71952 22.634131 0, 108.677606 22.837856 0, 108.280301 22.765174 0, 107.852309 22.685867 0, 107.893614 22.491012 0)))


## Search ASF Archive

In [8]:
results = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1],
                         processingLevel=asf.SLC,
                         beamMode=asf.BEAMMODE.IW,
                         relativeOrbit=relorb,
                         intersectsWith=str(myburst.geometry.values[0].centroid))
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)
for i in range(len(gf)):
    print(gf.geometry.values[i].intersection(myburst.geometry.values[0]).area)
    print(gf.fileID.values[i])

0.17001532095190802
S1A_IW_SLC__1SDV_20230911T105026_20230911T105053_050279_060D7B_D9AD-SLC
0.16990235038247864
S1A_IW_SLC__1SDV_20230830T105025_20230830T105052_050104_060788_BD25-SLC
0.16987844751515957
S1A_IW_SLC__1SDV_20230818T105025_20230818T105052_049929_060183_8094-SLC
0.16948882858863903
S1A_IW_SLC__1SDV_20230607T105020_20230607T105047_048879_05E0C5_86E6-SLC
0.16989695805024435
S1A_IW_SLC__1SDV_20230526T105020_20230526T105047_048704_05DB84_7AA4-SLC
0.17013944753286112
S1A_IW_SLC__1SDV_20230514T105019_20230514T105046_048529_05D65C_EA5C-SLC
0.16998937605413253
S1A_IW_SLC__1SDV_20230502T105019_20230502T105046_048354_05D0CA_6E49-SLC
0.16947931022382598
S1A_IW_SLC__1SDV_20230420T105018_20230420T105044_048179_05CAE5_9509-SLC
0.17014010012290062
S1A_IW_SLC__1SDV_20230408T105018_20230408T105045_048004_05C508_A1DF-SLC
0.1701252247585071
S1A_IW_SLC__1SDV_20230327T105018_20230327T105044_047829_05BF1F_300D-SLC
0.16966386196076585
S1A_IW_SLC__1SDV_20230315T105017_20230315T105044_047654_05B93

In [9]:
# 1st acquisition of every month
#gf.groupby([gf.index.year, gf.index.month]).first()

# 1st acquisition of given month every year
subset = gf#[gf.index.month == 1] 
subset = subset#.groupby(subset.index.year).first()

In [10]:
subset

,geometry,beamModeType,browse,bytes,centerLat,centerLon,faradayRotation,fileID,flightDirection,groupID,...,processingDate,processingLevel,sceneName,sensor,startTime,stopTime,url,pgeVersion,fileName,frameNumber
0,"POLYGON ((107.82407 22.83415, 108.16798 21.206...",IW,None,4814738884,22.2357,109.2019,None,S1A_IW_SLC__1SDV_20230911T105026_20230911T1050...,ASCENDING,S1A_IWDV_0067_0073_050279_157,...,2023-09-11T10:50:26.509Z,SLC,S1A_IW_SLC__1SDV_20230911T105026_20230911T1050...,C-SAR,2023-09-11T10:50:26.509Z,2023-09-11T10:50:53.456Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230911T105026_20230911T1050...,67
1,"POLYGON ((107.82461 22.83430, 108.16843 21.207...",IW,None,4757782402,22.2361,109.2025,None,S1A_IW_SLC__1SDV_20230830T105025_20230830T1050...,ASCENDING,S1A_IWDV_0067_0073_050104_157,...,2023-08-30T10:50:25.857Z,SLC,S1A_IW_SLC__1SDV_20230830T105025_20230830T1050...,C-SAR,2023-08-30T10:50:25.857Z,2023-08-30T10:50:52.797Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230830T105025_20230830T1050...,67
2,"POLYGON ((107.82455 22.83518, 108.16840 21.207...",IW,None,4755899601,22.2368,109.2024,None,S1A_IW_SLC__1SDV_20230818T105025_20230818T1050...,ASCENDING,S1A_IWDV_0067_0073_049929_157,...,2023-08-18T10:50:25.325Z,SLC,S1A_IW_SLC__1SDV_20230818T105025_20230818T1050...,C-SAR,2023-08-18T10:50:25.325Z,2023-08-18T10:50:52.269Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230818T105025_20230818T1050...,67
3,"POLYGON ((107.82676 22.83420, 108.17065 21.206...",IW,None,4819375545,22.2365,109.2089,None,S1A_IW_SLC__1SDV_20230607T105020_20230607T1050...,ASCENDING,S1A_IWDV_0067_0073_048879_157,...,2023-06-07T10:50:20.000Z,SLC,S1A_IW_SLC__1SDV_20230607T105020_20230607T1050...,C-SAR,2023-06-07T10:50:20.000Z,2023-06-07T10:50:47.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230607T105020_20230607T1050...,67
4,"POLYGON ((107.82468 22.83414, 108.16857 21.206...",IW,None,4780138661,22.2365,109.2068,None,S1A_IW_SLC__1SDV_20230526T105020_20230526T1050...,ASCENDING,S1A_IWDV_0067_0073_048704_157,...,2023-05-26T10:50:20.000Z,SLC,S1A_IW_SLC__1SDV_20230526T105020_20230526T1050...,C-SAR,2023-05-26T10:50:20.000Z,2023-05-26T10:50:47.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230526T105020_20230526T1050...,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,"POLYGON ((107.64651 23.66712, 107.99189 22.040...",IW,None,4863603761,23.0676,109.0288,None,S1A_IW_SLC__1SDV_20151023T104952_20151023T1050...,ASCENDING,S1A_IWDV_0071_0076_008279_157,...,2015-10-23T10:49:52.000Z,SLC,S1A_IW_SLC__1SDV_20151023T104952_20151023T1050...,C-SAR,2015-10-23T10:49:52.000Z,2015-10-23T10:50:19.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,002.53,S1A_IW_SLC__1SDV_20151023T104952_20151023T1050...,70
292,"POLYGON ((107.64668 23.66789, 107.99200 22.040...",IW,None,4888094116,23.0688,109.0315,None,S1A_IW_SLC__1SDV_20150929T104952_20150929T1050...,ASCENDING,S1A_IWDV_0071_0076_007929_157,...,2015-09-29T10:49:52.000Z,SLC,S1A_IW_SLC__1SDV_20150929T104952_20150929T1050...,C-SAR,2015-09-29T10:49:52.000Z,2015-09-29T10:50:18.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,002.53,S1A_IW_SLC__1SDV_20150929T104952_20150929T1050...,70
293,"POLYGON ((107.64615 23.66647, 107.99147 22.039...",IW,None,4866943255,23.0675,109.0310,None,S1A_IW_SLC__1SDV_20150812T104950_20150812T1050...,ASCENDING,S1A_IWDV_0071_0076_007229_157,...,2015-08-12T10:49:50.000Z,SLC,S1A_IW_SLC__1SDV_20150812T104950_20150812T1050...,C-SAR,2015-08-12T10:49:50.000Z,2015-08-12T10:50:17.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,002.53,S1A_IW_SLC__1SDV_20150812T104950_20150812T1050...,70
294,"POLYGON ((107.64619 23.66755, 107.99150 22.040...",IW,None,4878493302,23.0685,109.0310,None,S1A_IW_SLC__1SDV_20150719T104948_20150719T1050...,ASCENDING,S1A_IWDV_0071_0076_006879_157,...,2015-07-19T10:49:48.000Z,SLC,S1A_

In [11]:
subset.reset_index(drop=True).set_crs(4326).explore(column='startTime', style_kwds=dict(fillOpacity=.3))

/home/jovyan/.local/envs/hyp3-isce2/lib/python3.11/site-packages/geopandas/explore.py:400: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(gdf[column]):


In [12]:
# Get IPF version for each of these
def get_ipf(sceneName):
    sat = sceneName[0]+sceneName[2]
    url = f'https://datapool.asf.alaska.edu/METADATA_SLC/{sat}/{sceneName}.iso.xml'
    print(url)
    r = requests.get(url)
    
    IPF = re.search(r'\(version(.*?)\)',r.text).group(1).strip()
    
    return IPF

In [13]:
#subset['IPF'] = subset.sceneName.apply(get_ipf)

In [14]:
#subset.IPF

In [15]:
# Only IPF>=3.4 has burstnumbers in the metadata, others require reverse-lookup based on TANX
def get_metadata_xml(session, params):
    root = hb.download_metadata(session, params)  
    
    return root

def get_ipf(root):
    ''' for consolidated XML metadata get version '''
    ipfnode = root.find('.//safe:software', {'safe':'http://www.esa.int/safe/sentinel-1.0'})
    
    return ipfnode.attrib['version']


def get_burstnumber(session, row, myburst, product_schema='./support/s1-level-1-product.xsd'):
    ''' given an ASF frame, determine relative burst number corresponding to standard esa burstid
    
    session = asf session from hyp3-isce2 
    row = geodataframe row of asf_search results
    myburst = geoseries for burst of interest
    
    Note: requires support/s1-level-1-product.xsd XML schema from SLC SAFE for parsing metadata
    '''
    params = hb.BurstParams(row.sceneName, myburst.subswath_name, row.polarization[:2], 1)
    
    # Get All XML metadata for SLC
    root = get_metadata_xml(session, params)
    #print(root)
    IPF = get_ipf(root)
    #print(IPF)

    # Extract correct section of xml
    for product in root.findall('.//product'):
        prod_pol = product.find('polarisation').text
        prod_swath = product.find('swath').text
        
        if (prod_pol == params.polarization) and (prod_swath == params.swath):
            node = product.find('content')
            node.tag = 'product'
            string = lxml.etree.tostring(node, encoding='unicode')
            
    # Convert to python dictionary
    xs = xmlschema.XMLSchema(product_schema)
    parsed = xs.to_dict(string, validation='lax')[0]
    
    if IPF >= '999.9':
        burstid = [t.get('burstId').get('$') for t in parsed['swathTiming']['burstList']['burst']]
        burstnum = burstid.index(myburst.burst_id)
    else:
        tanx = np.array([t['azimuthAnxTime'] for t in parsed['swathTiming']['burstList']['burst']])
        print(tanx)
        burstnum = np.argmin(np.abs(tanx - myburst.time_from_anx_sec))
        print(myburst.time_from_anx_sec)
        if np.abs(tanx[burstnum]-myburst.time_from_anx_sec)>1.5:
            burstnum=-1
    return burstnum

In [16]:
myburst # '137_IW2_292399'

,burst_id,subswath_name,relative_orbit_number,time_from_anx_sec,orbit_pass,geometry
3,335208,IW1,157,361.558646,ASCENDING,"MULTIPOLYGON Z (((107.89361 22.49101 0.00000, ..."


In [17]:
asf_session = hb.get_asf_session()

In [18]:
# For small number of granules just iterate over pandas dataframe (slow but works)
Bursts = []
for i,row in subset.iterrows():
    num = get_burstnumber(asf_session, row, myburst.iloc[0])
    Bursts.append(num)
    print(num)
    #num

[341.14940411 343.90384955 346.65829499 349.41479599 352.17335255
 354.9319091  357.69046566 360.45107777 363.20757876]
361.5586459999941
7
[341.16065533 343.91304522 346.66749066 349.42604722 352.18254821
 354.94110477 357.69966132 360.45821788 363.21471888]
361.5586459999941
7
[341.17115189 343.92354178 346.67798722 349.43448822 352.19304477
 354.95160133 357.71015788 360.46871444 363.22727099]
361.5586459999941
7
[341.15220322 343.90253755 346.65698299 349.41348399 352.17204055
 354.9305971  357.68915366 360.44771021 363.20626676]
361.5586459999941
7
[341.15727445 343.90555322 346.65999866 349.4144441  352.17300066
 354.93155721 357.69011377 360.44867032 363.20722688]
361.5586459999941
7
[341.15211578 343.90245011 346.65689555 349.41339655 352.1719531
 354.93050966 357.68906621 360.44556721 363.20412376]
361.5586459999941
7
[341.14786511 343.90642166 346.65881155 349.4173681  352.1738691
 354.93242566 357.69098221 360.44953877 363.20603976]
361.5586459999941
7
[341.14792711 343.9126

In [19]:
subset['burst'] = Bursts

In [20]:
subset['date'] = gpd.pd.to_datetime(subset.startTime).dt.strftime('%Y%m%d')

In [21]:
with gpd.pd.option_context('display.max_colwidth', None):
    display(subset.loc[:,['date','sceneName','burst']])
    print(subset.loc[:,['burst']])

,date,sceneName,burst
0,20230911,S1A_IW_SLC__1SDV_20230911T105026_20230911T105053_050279_060D7B_D9AD,7
1,20230830,S1A_IW_SLC__1SDV_20230830T105025_20230830T105052_050104_060788_BD25,7
2,20230818,S1A_IW_SLC__1SDV_20230818T105025_20230818T105052_049929_060183_8094,7
3,20230607,S1A_IW_SLC__1SDV_20230607T105020_20230607T105047_048879_05E0C5_86E6,7
4,20230526,S1A_IW_SLC__1SDV_20230526T105020_20230526T105047_048704_05DB84_7AA4,7
...,...,...,...
291,20151023,S1A_IW_SLC__1SDV_20151023T104952_20151023T105019_008279_00BAB0_96F8,2
292,20150929,S1A_IW_SLC__1SDV_20150929T104952_20150929T105018_007929_00B137_58F2,2
293,20150812,S1A_IW_SLC__1SDV_20150812T104950_20150812T105017_007229_009E40_FDC9,2
294,20150719,S1A_IW_SLC__1SDV_20150719T104948_20150719T105015_006879_009495_3452,2


     burst
0        7
1        7
2        7
3        7
4        7
..     ...
291      2
292      2
293      2
294      2
295      2

[296 rows x 1 columns]


In [22]:
# Get a summer scene from each year 
gf = gf.set_index(gpd.pd.to_datetime(gf.startTime))
gf.head()


,geometry,beamModeType,browse,bytes,centerLat,centerLon,faradayRotation,fileID,flightDirection,groupID,...,sceneName,sensor,startTime,stopTime,url,pgeVersion,fileName,frameNumber,burst,date
startTime,,,,,,,,,,,,,,,,,,,,,
2023-09-11 10:50:26.509000+00:00,"POLYGON ((107.82407 22.83415, 108.16798 21.206...",IW,None,4814738884,22.2357,109.2019,None,S1A_IW_SLC__1SDV_20230911T105026_20230911T1050...,ASCENDING,S1A_IWDV_0067_0073_050279_157,...,S1A_IW_SLC__1SDV_20230911T105026_20230911T1050...,C-SAR,2023-09-11T10:50:26.509Z,2023-09-11T10:50:53.456Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230911T105026_20230911T1050...,67,7,20230911
2023-08-30 10:50:25.857000+00:00,"POLYGON ((107.82461 22.83430, 108.16843 21.207...",IW,None,4757782402,22.2361,109.2025,None,S1A_IW_SLC__1SDV_20230830T105025_20230830T1050...,ASCENDING,S1A_IWDV_0067_0073_050104_157,...,S1A_IW_SLC__1SDV_20230830T105025_20230830T1050...,C-SAR,2023-08-30T10:50:25.857Z,2023-08-30T10:50:52.797Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230830T105025_20230830T1050...,67,7,20230830
2023-08-18 10:50:25.325000+00:00,"POLYGON ((107.82455 22.83518, 108.16840 21.207...",IW,None,4755899601,22.2368,109.2024,None,S1A_IW_SLC__1SDV_20230818T105025_20230818T1050...,ASCENDING,S1A_IWDV_0067_0073_049929_157,...,S1A_IW_SLC__1SDV_20230818T105025_20230818T1050...,C-SAR,2023-08-18T10:50:25.325Z,2023-08-18T10:50:52.269Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230818T105025_20230818T1050...,67,7,20230818
2023-06-07 10:50:20+00:00,"POLYGON ((107.82676 22.83420, 108.17065 21.206...",IW,None,4819375545,22.2365,109.2089,None,S1A_IW_SLC__1SDV_20230607T105020_20230607T1050...,ASCENDING,S1A_IWDV_0067_0073_048879_157,...,S1A_IW_SLC__1SDV_20230607T105020_20230607T1050...,C-SAR,2023-06-07T10:50:20.000Z,2023-06-07T10:50:47.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230607T105020_20230607T1050...,67,7,20230607
2023-05-26 10:50:20+00:00,"POLYGON ((107.82468 22.83414, 108.16857 21.206...",IW,None,4780138661,22.2365,109.2068,None,S1A_IW_SLC__1SDV_20230526T105020_20230526T1050...,ASCENDING,S1A_IWDV_0067_0073_048704_157,...,S1A_IW_SLC__1SDV_20230526T105020_20230526T1050...,C-SAR,2023-05-26T10:50:20.000Z,2023-05-26T10:50:47.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,003.61,S1A_IW_SLC__1SDV_20230526T105020_20230526T1050...,67,7,20230526


In [23]:
with open(f'../stack.txt', 'w') as f:
    f.write('')
for i in range(len(subset)):
    cmd = str(subset.iloc[i].sceneName)+','+subswath+','+str(subset.iloc[i].burst)+'\n'
    if str(subset.iloc[i].burst) !='-1':
        with open(f'../process/data/stack.txt', 'a') as f:
            f.write(cmd)
    else:
        print('跳过')

跳过
